In [1]:
import pandas as pd
import string,re
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional,LSTM,Concatenate,Dropout
from tensorflow.keras.regularizers import l2

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

## Preprocess The Text

In [2]:
data=pd.read_csv("/kaggle/input/headlinegen/inshort.csv")
data['Short']=data['Short'].str.strip()
data['Headline']=data['Headline'].str.strip()
data.head()

,Short,Headline
0,The CBI on Saturday booked four former officia...,4 ex-bank officials booked for cheating bank o...
1,Chief Justice JS Khehar has said the Supreme C...,Supreme Court to go paperless in 6 months: CJI
2,"At least three people were killed, including a...","At least 3 killed, 30 injured in blast in Sylh..."
3,Mukesh Ambani-led Reliance Industries (RIL) wa...,Why has Reliance been barred from trading in f...
4,TV news anchor Arnab Goswami has said he was t...,Was stopped from entering my own studio at Tim...


## clean the data

In [3]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import nltk
import re

def clean_text(text):
    # Use BeautifulSoup to remove HTML character codes from the text
    text = BeautifulSoup(text, 'html.parser').get_text()
    text = re.sub(r'\d+', '', text)
    # Use a regular expression to split the text into words
    words = re.findall(r'\b\w+\b', text)

    # Initialize an empty list to store the cleaned words
    cleaned_words = []

    # Iterate over the words in the text
    for word in words:
        # Check if the word is a stopword
        if word.lower() not in stopwords.words('english'):
            # If the word is not a stopword, add it to the list of cleaned words
            cleaned_words.append(word)

    # Join the cleaned words back into a single string
    cleaned_text = ' '.join(cleaned_words)

    return cleaned_text

#sample_text = "A new trailer for the upcoming superhero film 'Justice League' was released on Saturday. Based on the DC Comics superhero team, the film stars Ben Affleck as 'Batman', Gal Gadot as 'Wonder Woman', Ezra Miller as 'The Flash' and Jason Momoa as 'Aquaman'. Directed by Zack Snyder, the film is scheduled to release on November 17, 2017."

# Preprocess the 'Short' and 'Headline' columns
data['Short'] = data['Short'].apply(clean_text)
data['Headline'] = data['Headline'].apply(clean_text)
data['Headline']='<s> '+data['Headline']+' <e>'
# Display the preprocessed data


/tmp/ipykernel_29/1779599224.py:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()


In [4]:
data.head(11)

,Short,Headline
0,CBI Saturday booked four former officials Synd...,<s> ex bank officials booked cheating bank cro...
1,Chief Justice JS Khehar said Supreme Court go ...,<s> Supreme Court go paperless months CJI <e>
2,least three people killed including policeman ...,<s> least killed injured blast Sylhet Banglade...
3,Mukesh Ambani led Reliance Industries RIL barr...,<s> Reliance barred trading futures <e>
4,TV news anchor Arnab Goswami said told could p...,<s> stopped entering studio Times Arnab <e>
5,new trailer upcoming superhero film Justice Le...,<s> New trailer Justice League released <e>
6,Television actress Shilpa Shinde opening claim...,<s> touch right Shilpa Shinde sexual harassmen...
7,Uttar Pradesh Chief Minister Yogi Adityanath v...,<s> Anti Romeo squads must trouble consenting ...
8,apparent jibe anti Romeo squads Delhi Tourism ...,<s> Romeo Juliet welcome Delhi AAP Minister <e>
9,least people reportedly injured blasts occurre...,<s> blasts occur ordnance factory MP injured <e>


In [5]:
short=data['Short']
head=data['Headline']
short=short.str.lower()
head=head.str.lower()

## Tokenization

In [6]:
uniq_word_short=set()
uniq_word_head=set()

count=0
for i in short:
    for j in i.split():
        uniq_word_short.add(j)
print(len(uniq_word_short))
for i in head:
    for j in i.split():
        uniq_word_head.add(j)
print(len(uniq_word_head))
uniq_word_short=list(uniq_word_short)
uniq_word_head=list(uniq_word_head)

65483
26512


In [7]:
word_ind_train={}
ind_word_train={}
word_ind_target={}
ind_word_target={}

# for encoder input 
for i in range(1,len(uniq_word_short)+1):
    word_ind_train[uniq_word_short[i-1]]=i
for i in range(1,len(uniq_word_short)+1):
    ind_word_train[i]=uniq_word_short[i-1]
    
# for decoder input
for i in range(1,len(uniq_word_head)+1):
    word_ind_target[uniq_word_head[i-1]]=i
for i in range(1,len(uniq_word_head)+1):
    ind_word_target[i]=uniq_word_head[i-1]

## Generate Training and Testing data

In [8]:
train_data=[]
target_data=[]

for i in short:
    w_i=[]
    for j in i.split():
        w_i.append(word_ind_train[j])
    train_data.append(w_i)
        
for i in head:
    i_w=[]
    for j in i.split():
        i_w.append(word_ind_target[j])
    target_data.append(i_w)
    

In [9]:
# Pad encoded sentences
max_encoder_seq_length = max([len(seq) for seq in train_data])
max_decoder_seq_length = max([len(seq) for seq in target_data])
print(max_encoder_seq_length,max_decoder_seq_length)

61 14


In [10]:
encoder_input_data = pad_sequences(train_data,maxlen=max_encoder_seq_length, padding='post')
decoder_input_data = pad_sequences(target_data,maxlen=max_decoder_seq_length, padding='post')

# target data for the decoder
decoder_target_data = []
for seq in target_data:
    decoder_target_data.append(seq[1:])
decoder_target_data = pad_sequences(decoder_target_data,maxlen=max_decoder_seq_length, padding='post')
num_decoder_tokens = len(word_ind_target) + 1
# decoder_output = np.zeros((len(target_data), max_decoder_seq_length,num_decoder_tokens), dtype='float32')
# for i, seq in enumerate(decoder_target_data):
#     for t, token in enumerate(seq):
#         decoder_output[i, t, token] = 1

## Arhitecture Encoder(Bi-directional)-Decoder(vanilla Lstm)

In [11]:
# Define the model parameters
latent_dim = 256
num_encoder_tokens = len(word_ind_train) + 1
num_decoder_tokens = len(word_ind_target) + 1

# Create the embedding layers
encoder_embedding_layer = Embedding(num_encoder_tokens, latent_dim)
decoder_embedding_layer = Embedding(num_decoder_tokens, latent_dim)

# Define the encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = encoder_embedding_layer(encoder_inputs)
encoder_outputs, forward_h, backward_h, forward_c, backward_c = Bidirectional(LSTM(latent_dim, return_state=True, kernel_regularizer=l2(0.01)))(encoder_embedding)
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]

# Define the decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = decoder_embedding_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim * 2, return_sequences=True, return_state=True, kernel_regularizer=l2(0.01))
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
decoder_outputs = Dropout(0.1)(decoder_outputs)

# Create the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 256)    16763904    ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 bidirectional (Bidirectional)  [(None, 512),        1050624     ['embedding[0][0]']              
                                 (None, 256),                                                 

## Genearte data in batch

In [13]:
import numpy as np
from keras.utils import Sequence

class DataGenerator(Sequence):
    def __init__(self, input_data, output_data, target_data, batch_size=32):
        self.input_data = input_data
        self.output_data = output_data
        self.target_data = target_data
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.input_data) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_input = self.input_data[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_output = self.output_data[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_target = self.target_data[idx * self.batch_size:(idx + 1) * self.batch_size]
        return [batch_input, batch_output], batch_target


## split into encoder-decoder input-output

In [14]:
# Define the batch size and number of epochs
batch_size = 32
epochs = 60

from sklearn.model_selection import train_test_split

# Split data into training and testing sets
encoder_input_train, encoder_input_test, \
decoder_input_train, decoder_input_test, \
decoder_target_train, decoder_target_test = train_test_split(
    encoder_input_data, decoder_input_data, decoder_target_data, test_size=0.1, random_state=42)

# Create the data generators
train_data = DataGenerator(encoder_input_train, decoder_input_train, decoder_target_train, batch_size=batch_size)
test_data = DataGenerator(encoder_input_test, decoder_input_test, decoder_target_test, batch_size=batch_size)

## Training

In [21]:
# Compile and train the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Train the model using the data generator
history = model.fit(train_data, validation_data=test_data, epochs=20)

Epoch 1/20
1550/1550 [==============================] - 90s 54ms/step - loss: 3.0744 - accuracy: 0.6086 - val_loss: 4.0370 - val_accuracy: 0.5643
Epoch 2/20
1550/1550 [==============================] - 61s 40ms/step - loss: 2.9675 - accuracy: 0.6235 - val_loss: 4.0914 - val_accuracy: 0.5635
Epoch 3/20
1550/1550 [==============================] - 62s 40ms/step - loss: 2.8737 - accuracy: 0.6328 - val_loss: 4.1189 - val_accuracy: 0.5658
Epoch 4/20
1550/1550 [==============================] - 62s 40ms/step - loss: 2.8126 - accuracy: 0.6402 - val_loss: 4.1457 - val_accuracy: 0.5650
Epoch 5/20
1550/1550 [==============================] - 61s 40ms/step - loss: 2.7601 - accuracy: 0.6477 - val_loss: 4.1855 - val_accuracy: 0.5624
Epoch 6/20
1550/1550 [==============================] - 61s 40ms/step - loss: 2.7014 - accuracy: 0.6551 - val_loss: 4.2128 - val_accuracy: 0.5632
Epoch 7/20
1550/1550 [==============================] - 61s 40ms/step - loss: 2.6615 - accuracy: 0.6608 - val_loss: 4.2432 -

In [22]:
loss, accuracy = model.evaluate(test_data)

173/173 [==============================] - 3s 19ms/step - loss: 4.5851 - accuracy: 0.5625


## Inference of encoder-decoder

In [23]:
# Define the encoder model
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.save('encoder_head.h5')

# Define the decoder model
decoder_state_input_h = Input(shape=(latent_dim * 2,))
decoder_state_input_c = Input(shape=(latent_dim * 2,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
decoder_model.save('decoder_head.h5')

In [24]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = word_ind_target['<s>']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = ind_word_target[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '<e>' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence


## Testing on some sentence

In [27]:
for i in range(20,41):
    new_english_sentence=short[i]
    print("-----------------------")
    print(short[i])
    new_english_sentence=new_english_sentence.strip()
#     new_english_sentence.lower().replace('[^\w\s\']', '')
#     new_english_sentence.replace(r'(?<=[a-zA-Z])\d+|\d+(?=[a-zA-Z])', '')
    #new_eng_integer_encoded=eng_token.texts_to_sequences([new_english_sentence])
    input_tokens=[]
    for i in new_english_sentence.split():
        input_tokens.append(word_ind_train[i])
    print(input_tokens)
    new_encoder_input_data=pad_sequences([input_tokens],maxlen=max_encoder_seq_length, padding='post')
    decoded_sentence=decode_sequence(new_encoder_input_data)
    decoded_sentence=decoded_sentence.strip()
    # print('Input sentence:', new_english_sentence)
    #print('Actual sentence:', short[i])
    print('Decoded sentence:', decoded_sentence)
    print("------------------------")

-----------------------
indian army consultations defence ministry considering cutting number sahayaks translates jawans comes videos soldiers alleging often made perform menial tasks officers surfaced online sahayak jawans moved army formations replaced civilians
[1607, 1280, 49522, 2229, 39731, 41314, 21322, 16031, 54936, 43664, 64706, 45425, 19220, 25480, 20851, 40161, 27732, 16242, 10757, 62396, 46703, 28277, 39608, 52411, 64706, 58301, 1280, 56025, 47635, 62501]
1/1 [==============================] - 0s 20ms/step
Decoded sentence: pak army claims
------------------------
-----------------------
police officer lucknow nagesh mishra suspended duty made everyone swear vow cleanliness consuming tobacco notably uttar pradesh cm yogi adityanath directed officials consuming pan masala office premises uttar pradesh cops suspended discipline related issues ever since adityanath assumed office
[41641, 34004, 57345, 22754, 2309, 22573, 63397, 27732, 40399, 50230, 32660, 56944, 6940, 23696, 6

In [29]:
import json

in_word_train = {"word_index": ind_word_train}
with open('in_word_train.json', 'w') as f:
    json.dump(in_word_train, f)
in_word_target = {"word_index": ind_word_target}
with open('in_word_target.json', 'w') as f:
    json.dump(in_word_target, f)
    
    
word_in_train = {"word_index": word_ind_train}
with open('word_in_train.json', 'w') as f:
    json.dump(word_in_train, f)
word_in_target = {"word_index": word_ind_target}
with open('word_in_target.json', 'w') as f:
    json.dump(word_in_target, f)
